***Посмотрим какие жанры наиболее выпускаемые и используем их для сбора данных***

In [ ]:
import requests
import json
import csv
import numpy as np
import pandas as pd

In [1]:
API_key = "d081a22bc7714c7f9d5d381403505669"

url = f"https://api.rawg.io/api/genres"
resp = requests.get(url, params={"key": API_key}, timeout=10)

if resp.status_code != 200:
    print("Ошибка:", resp.status_code)
    print(resp.text[:500])
else:
    data = resp.json()


In [2]:
d=dict()
for g in data.get("results", []):
    name = g.get("name")
    count = g.get("games_count")
    d[name]= count
    print(f"{name} - {count}")

Action - 190929
Indie - 85394
Adventure - 151116
RPG - 61659
Strategy - 62134
Shooter - 59603
Casual - 66992
Simulation - 76455
Puzzle - 97372
Arcade - 22672
Platformer - 100905
Massively Multiplayer - 4224
Racing - 25726
Sports - 22562
Fighting - 11773
Family - 5409
Board Games - 8389
Card - 4540
Educational - 15719


In [3]:
df = pd.DataFrame.from_dict(d, orient='index', columns=['games_count']).reset_index()
df = df.rename(columns={'index': 'genre'})
df= df.sort_values('games_count', ascending=False).reset_index().drop(columns='index')
df

,genre,games_count
0,Action,190929
1,Adventure,151116
2,Platformer,100905
3,Puzzle,97372
4,Indie,85394
5,Simulation,76455
6,Casual,66992
7,Strategy,62134
8,RPG,61659
9,Shooter,59603


***Посмотрим на то, какие id у платформ чтобы и также по аналогии с первым пунктом сделать выборку по популярной платорме***

In [4]:
import requests
import json
API_key = "d081a22bc7714c7f9d5d381403505669"

url = f"https://api.rawg.io/api/platforms"
resp = requests.get(url, params={"key": API_key}, timeout=10)

if resp.status_code != 200:
    print("Ошибка:", resp.status_code)
    print(resp.text[:500])
else:
    data = resp.json()

In [5]:
rows = []
for p in data.get("results", []):
    rows.append({
        "id": p.get("id"),
        "platform": p.get("name"),
        "games_count": p.get("games_count")
    })

df = pd.DataFrame(rows)
df = df.sort_values("games_count", ascending=False).reset_index(drop=True)
df

,id,platform,games_count
0,4,PC,559070
1,5,macOS,107814
2,6,Linux,80202
3,3,iOS,77418
4,21,Android,52488
5,18,PlayStation 4,6948
6,7,Nintendo Switch,5718
7,1,Xbox One,5713
8,16,PlayStation 3,3165
9,14,Xbox 360,2808


# Набираем данные

In [6]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=action&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),action.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            # защититься от None
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1

            # если встретили игру с rating == 0 -> завершаем парсинг, чтобы не захватывать нерелевантные игры
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break

        # перейти к следующей странице, если есть next
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")


830060 | Sonic Triple Trouble 16-Bit (NoahNCopeland) |  Indie, Platformer, Adventure, Action | released: 2022-08-02 | meta: nan | rating: 4.83 | platforms: PC
4167 | Mass Effect Trilogy |  Shooter, Action, RPG | released: 2012-11-06 | meta: nan | rating: 4.75 | platforms: PC, PlayStation 3
605544 | Bloodborne: The Old Hunters |  Action, RPG | released: 2015-11-24 | meta: nan | rating: 4.74 | platforms: PlayStation 4
13370 | Red Rope: Don't Fall Behind |  Indie, Adventure, Action | released: 2016-07-21 | meta: nan | rating: 4.71 | platforms: PC, macOS, Linux
111 | Forgotten Memories: Remastered Edition |  Indie, Adventure, Action, Simulation | released: 2015-04-23 | meta: 72 | rating: 4.71 | platforms: PC, Nintendo Switch, iOS, Android
309793 | Micro Mages |  Indie, Action | released: 2019-05-01 | meta: nan | rating: 4.71 | platforms: PC, NES
846303 | Cyberpunk 2077: Phantom Liberty |  Shooter, Action | released: 2023-09-26 | meta: nan | rating: 4.71 | platforms: PC, PlayStation 5, Xbox

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=adventure&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),adventure.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")


            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

Выходные данные были обрезаны до нескольких последних строк (5000).
965927 | Corrupting The Universe |  Adventure | released: 2023-09-04 | meta: nan | rating: 4.5 | platforms: PC, Android, macOS, Linux
487569 | Henry Stickmin Collection - Full Game |  Strategy, Adventure, Action | released: 2020-08-29 | meta: nan | rating: 4.5 | platforms: PC, macOS, Linux
12203 | Pocket Kingdom |  Indie, Adventure | released: 2016-12-08 | meta: nan | rating: 4.5 | platforms: PC, macOS, Linux
974381 | Snakebird Complete |  Adventure, Puzzle | released: 2023-12-29 | meta: nan | rating: 4.5 | platforms: PC, macOS
27985 | Skylanders SuperChargers |  Adventure, Action | released: 2015-09-20 | meta: nan | rating: 4.5 | platforms: Xbox One, PlayStation 4, iOS, Xbox 360, PlayStation 3, Wii U
632452 | Dream SMP: Las Nevadas Bedrock |  Adventure | released: 2021-07-08 | meta: nan | rating: 4.5 | platforms: PC, Android, macOS, Linux
743538 | Aurelia |  Indie, Adventure, RPG | released: 2021-12-04 | meta: nan | r

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=platformer&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),platformer.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

830060 | Sonic Triple Trouble 16-Bit (NoahNCopeland) |  Indie, Platformer, Adventure, Action | released: 2022-08-02 | meta: nan | rating: 4.83 | platforms: PC
24277 | Aliens: Infestation |  Shooter, Platformer | released: 2011-10-11 | meta: nan | rating: 4.57 | platforms: Nintendo DS
419292 | I Wanna Be the Boshy |  Platformer | released: 2010-11-02 | meta: nan | rating: 4.56 | platforms: PC
9819 | Shovel Knight: Specter of Torment |  Indie, Platformer, Adventure, Action | released: 2017-03-03 | meta: nan | rating: 4.54 | platforms: PC, PlayStation 4, Nintendo Switch, Nintendo 3DS, macOS, Linux
254545 | Cuphead: The Delicious Last Course |  Platformer | released: 2022-06-30 | meta: 89 | rating: 4.51 | platforms: PC, Xbox One, PlayStation 4, Nintendo Switch
989331 | Legacy of Kain™ Soul Reaver 1&2 Remastered |  Platformer, Adventure, Action, Puzzle | released: 2024-12-10 | meta: nan | rating: 4.5 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
4

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=puzzle&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),puzzle.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")

            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

90776 | Hello Charlotte EP2: Requiem Aeternam Deo |  Indie, Adventure, RPG, Puzzle | released: 2016-09-22 | meta: nan | rating: 4.67 | platforms: PC
692717 | Death Palette |  Adventure, Puzzle | released: 2020-02-19 | meta: nan | rating: 4.67 | platforms: iOS
521769 | One Night at Flumpty's |  Strategy, Puzzle | released: 2020-11-18 | meta: nan | rating: 4.67 | platforms: iOS
4200 | Portal 2 |  Shooter, Puzzle | released: 2011-04-18 | meta: 95 | rating: 4.58 | platforms: PC, Xbox One, macOS, Linux, Xbox 360, PlayStation 3
826890 | shapez |  Puzzle | released: 2020-06-06 | meta: nan | rating: 4.56 | platforms: PC, iOS, Android, Linux
27969 | The Legend of Zelda: Ocarina of Time 3D |  Adventure, RPG, Puzzle | released: 2011-06-19 | meta: 94 | rating: 4.55 | platforms: Nintendo 3DS
64982 | Forgotten Hill Mementoes |  Indie, Adventure, Puzzle | released: 2018-03-06 | meta: nan | rating: 4.5 | platforms: PC, macOS
52116 | Bring You Home |  Casual, Indie, Adventure, Puzzle | released: 2018-0

In [7]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=indie&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),indie.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

KeyboardInterrupt: 

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=simulation&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),simulation.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

974923 | Winter Memories |  Casual, Adventure, RPG, Simulation | released: 2024-01-05 | meta: nan | rating: 4.83 | platforms: PC
27612 | Harvest Moon: Friends of Mineral Town (Wii) |  Adventure, Simulation | released: 2015-06-11 | meta: nan | rating: 4.71 | platforms: Wii U
111 | Forgotten Memories: Remastered Edition |  Indie, Adventure, Action, Simulation | released: 2015-04-23 | meta: 72 | rating: 4.71 | platforms: PC, Nintendo Switch, iOS, Android
463689 | siren head simulator (mr bonnie) |  Simulation | released: 2020-07-06 | meta: nan | rating: 4.67 | platforms: PC, Linux
762725 | FNAF SANDBOX |  Simulation | released: 2022-04-02 | meta: nan | rating: 4.58 | platforms: PC
22594 | planetarian HD |  Casual, Adventure, Simulation | released: 2017-05-08 | meta: nan | rating: 4.57 | platforms: PC
232225 | Online Soccer Manager (OSM) |  Simulation, Sports | released: 2010-11-06 | meta: nan | rating: 4.56 | platforms: iOS, Android, Web
305984 | Rush Rally 3 |  Racing, Indie, Simulation,

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=strategy&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),strategy.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()

print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

41975 | Victoria 2: Heart of Darkness |  Strategy | released: 2013-04-16 | meta: nan | rating: 4.71 | platforms: PC
40828 | Starsector |  Strategy, RPG | released: 2013-04-26 | meta: nan | rating: 4.67 | platforms: PC
49840 | Dominions 5 - Warriors of the Faith |  Strategy, Indie | released: 2017-11-27 | meta: nan | rating: 4.67 | platforms: PC, macOS, Linux
521769 | One Night at Flumpty's |  Strategy, Puzzle | released: 2020-11-18 | meta: nan | rating: 4.67 | platforms: iOS
38946 | StarСraft II: Legacy of the Void |  Strategy | released: 2015-11-10 | meta: 88 | rating: 4.59 | platforms: PC, macOS
38945 | StarCraft II: Heart of the Swarm |  Strategy | released: 2013-03-12 | meta: nan | rating: 4.58 | platforms: PC, macOS
38067 | StarCraft II: Wings of Liberty |  Strategy | released: 2010-07-27 | meta: 93 | rating: 4.57 | platforms: PC, macOS
850683 | Octopath Traveler 2 |  Strategy | released: 2023-02-24 | meta: nan | rating: 4.55 | platforms: PC, PlayStation 5, PlayStation 4, Nintendo

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=shooter&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),shooter.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

4167 | Mass Effect Trilogy |  Shooter, Action, RPG | released: 2012-11-06 | meta: nan | rating: 4.75 | platforms: PC, PlayStation 3
846303 | Cyberpunk 2077: Phantom Liberty |  Shooter, Action | released: 2023-09-26 | meta: nan | rating: 4.7 | platforms: PC, PlayStation 5, Xbox Series S/X
799265 | The Last of Us Part I |  Shooter, Adventure, Action | released: 2022-09-02 | meta: nan | rating: 4.67 | platforms: PC, PlayStation 5
58823 | Splatoon 2: Octo Expansion |  Shooter, Action | released: 2018-06-12 | meta: 82 | rating: 4.62 | platforms: Nintendo Switch
4200 | Portal 2 |  Shooter, Puzzle | released: 2011-04-18 | meta: 95 | rating: 4.58 | platforms: PC, Xbox One, macOS, Linux, Xbox 360, PlayStation 3
24277 | Aliens: Infestation |  Shooter, Platformer | released: 2011-10-11 | meta: nan | rating: 4.57 | platforms: Nintendo DS
516111 | Mass Effect: Legendary Edition |  Shooter, Adventure, Action, RPG | released: 2021-05-14 | meta: 88 | rating: 4.57 | platforms: PC, PlayStation 5, Xbox O

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=racing&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),racing.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

305984 | Rush Rally 3 |  Racing, Indie, Simulation, Sports | released: 2019-03-27 | meta: nan | rating: 4.56 | platforms: PC, iOS, Android
404454 | Need for Speed: Most Wanted Black Edition |  Racing | released: 2015-08-31 | meta: nan | rating: 4.46 | platforms: PC, Xbox, PlayStation 2
23165 | Mario Kart 8 Deluxe |  Racing | released: 2017-04-27 | meta: 92 | rating: 4.45 | platforms: Nintendo Switch
246761 | FR LEGENDS |  Racing, Action, Sports | released: 2018-09-27 | meta: nan | rating: 4.38 | platforms: iOS
45674 | DATA WING |  Casual, Racing, Arcade | released: 2017-08-16 | meta: nan | rating: 4.38 | platforms: iOS, Android
27976 | Mario Kart 8 |  Racing | released: 2014-05-29 | meta: 88 | rating: 4.37 | platforms: Wii U
881 | Forza Horizon 3 |  Racing, Arcade | released: 2016-09-27 | meta: 88 | rating: 4.37 | platforms: PC, Xbox One
58753 | Forza Horizon 4 |  Racing, Arcade | released: 2018-10-02 | meta: 90 | rating: 4.37 | platforms: PC, Xbox One, Xbox Series S/X
28637 | Forza Ho

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=sports&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),sports.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

232225 | Online Soccer Manager (OSM) |  Simulation, Sports | released: 2010-11-06 | meta: nan | rating: 4.56 | platforms: iOS, Android, Web
305984 | Rush Rally 3 |  Racing, Indie, Simulation, Sports | released: 2019-03-27 | meta: nan | rating: 4.56 | platforms: PC, iOS, Android
10321 | Racket: Nx |  Indie, Action, Sports | released: 2017-01-31 | meta: nan | rating: 4.5 | platforms: PC
490413 | Poker Club |  Casual, Strategy, Simulation, Sports, Indie | released: 2020-11-19 | meta: nan | rating: 4.5 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
61493 | Out of the Park Baseball 19 |  Strategy, Indie, Simulation, Sports | released: 2018-03-22 | meta: nan | rating: 4.45 | platforms: PC, macOS, Linux
61801 | PIPE by BMX Streets |  Indie, Simulation, Sports | released: 2018-03-15 | meta: nan | rating: 4.44 | platforms: PC, macOS
4262 | EA SPORTS FIFA Street |  Sports | released: 2012-09-04 | meta: nan | rating: 4.43 | platforms: Xbox 360, PlayStati

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=arcade&ordering=-rating"
k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),arcade.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break

        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break

        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break

        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

67281 | Arcaea |  Casual, Arcade | released: 2017-03-09 | meta: nan | rating: 4.71 | platforms: iOS
964507 | Geometry Dash RazorLeaf |  Casual, Indie, Arcade | released: 2023-07-19 | meta: nan | rating: 4.71 | platforms: PC, Android
140 | Sky Force 2014 |  Casual, Arcade, Action | released: 2014-04-26 | meta: nan | rating: 4.5 | platforms: iOS, Android
1524 | Love You To Bits |  Arcade, Adventure, Puzzle | released: 2016-02-25 | meta: nan | rating: 4.48 | platforms: PC, iOS, Android
791789 | Hatsune Miku: Project DIVA Mega Mix+ |  Arcade, Action | released: 2022-05-26 | meta: nan | rating: 4.47 | platforms: PC, Nintendo Switch
4534 | The Sly Collection |  Arcade, Adventure, Action | released: 2010-11-09 | meta: 85 | rating: 4.46 | platforms: PlayStation 3, PS Vita
1458 | Bug Princess |  Casual, Arcade, Action | released: 2011-12-15 | meta: 86 | rating: 4.46 | platforms: PC, Nintendo Switch, iOS
42337 | Shovel Knight: Plague of Shadows |  Arcade | released: 2015-09-17 | meta: 86 | ratin

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&genres=casual&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),casual.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

974923 | Winter Memories |  Casual, Adventure, RPG, Simulation | released: 2024-01-05 | meta: nan | rating: 4.83 | platforms: PC
743493 | Osake Riesz |  Casual | released: 2018-04-24 | meta: nan | rating: 4.83 | platforms: PC
67281 | Arcaea |  Casual, Arcade | released: 2017-03-09 | meta: nan | rating: 4.71 | platforms: iOS
964507 | Geometry Dash RazorLeaf |  Casual, Indie, Arcade | released: 2023-07-19 | meta: nan | rating: 4.71 | platforms: PC, Android
717427 | Athanasy |  Casual, Indie | released: 2022-04-13 | meta: nan | rating: 4.67 | platforms: PC, macOS, Linux
505805 | Shoot Shoot My Waifu |  Casual, Indie, Adventure, Action | released: 2020-10-12 | meta: nan | rating: 4.67 | platforms: PC, macOS, Linux
816336 | Desert Stalker (2022) |  Casual | released: 2022-04-17 | meta: nan | rating: 4.6 | platforms: PC
743549 | What a Legend! |  Casual | released: 2021-09-25 | meta: nan | rating: 4.58 | platforms: PC
22594 | planetarian HD |  Casual, Adventure, Simulation | released: 2017-0

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&platforms=187&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице
OUT_CSV = "rawg_games(2010-2025),ps5.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])
done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")
            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])

            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating

            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break

        page += 1

finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

257255 | The Witcher 3 Wild Hunt - Complete Edition |  Adventure, RPG | released: 2016-08-30 | meta: 92 | rating: 4.8 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
339958 | Persona 5 Royal |  Adventure, RPG | released: 2020-03-31 | meta: 94 | rating: 4.76 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
846303 | Cyberpunk 2077: Phantom Liberty |  Shooter, Action | released: 2023-09-26 | meta: nan | rating: 4.7 | platforms: PC, PlayStation 5, Xbox Series S/X
536886 | The Binding of Isaac: Repentance |  Action | released: 2021-03-31 | meta: nan | rating: 4.68 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
799265 | The Last of Us Part I |  Shooter, Adventure, Action | released: 2022-09-02 | meta: nan | rating: 4.67 | platforms: PC, PlayStation 5
545015 | Disco Elysium: Final Cut |  Adventure, RPG | released: 2021-03-30 | meta: 90 | rating: 4.66 | platforms: PC, P

In [ ]:
API_key = "d081a22bc7714c7f9d5d381403505669"
url = "https://api.rawg.io/api/games?dates=2010-01-01,2025-12-31&platforms=1&ordering=-rating"

k = 0
page = 1
page_size = 40     # сколько записей на странице

OUT_CSV = "rawg_games(2010-2025),xb1.csv"
f = open(OUT_CSV, "w", encoding="utf-8", newline="")
writer = csv.writer(f)
writer.writerow(["gid", "name", "genres", "released", "metacritic", "rating", "platforms"])

done = False
try:
    while True:
        resp = requests.get(url, params={"key": API_key, "page": page, "page_size": page_size}, timeout=15)
        if resp.status_code != 200:
            print("Ошибка:", resp.status_code)
            print(resp.text[:500])
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            print("Результаты закончились на странице", page)
            break
        for g in results:
            gid = g.get("id")
            name = g.get("name")
            released = g.get("released")
            metacritic = g.get("metacritic")
            rating = g.get("rating")

            platforms = [ (p.get("platform") or {}).get("name", "") for p in (g.get("platforms") or []) ]
            genres = ", ".join([x.get("name", "") for x in (g.get("genres") or [])])
            platforms_str = ", ".join([p for p in platforms if p])
            metacritic_val = np.nan if metacritic is None else metacritic
            rating_val = np.nan if rating is None else rating
            print(f"{gid} | {name} |  {genres} | released: {released} | meta: {metacritic_val} | rating: {rating_val} | platforms: {platforms_str}")
            writer.writerow([gid, name, genres, released, metacritic_val, rating_val, platforms_str])
            k += 1
            if rating_val == 0:
                print(f"Найдено ранжирование = 0 для gid={gid}. Останавливаем парсинг и сохраняем всё.")
                done = True
                break
        if done:
            break
        if not data.get("next"):
            print("Дальше страниц нет — заканчиваем.")
            break
        page += 1
finally:
    f.close()
print(f"Готово — обработано {k} игр. CSV -> {OUT_CSV}")

43252 | The Witcher 3: Wild Hunt – Blood and Wine |  RPG | released: 2016-05-30 | meta: 92 | rating: 4.81 | platforms: PC, Xbox One, PlayStation 4
257255 | The Witcher 3 Wild Hunt - Complete Edition |  Adventure, RPG | released: 2016-08-30 | meta: 92 | rating: 4.8 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
339958 | Persona 5 Royal |  Adventure, RPG | released: 2020-03-31 | meta: 94 | rating: 4.76 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
43050 | The Witcher 3: Wild Hunt – Hearts of Stone |  RPG | released: 2015-10-13 | meta: 90 | rating: 4.75 | platforms: PC, Xbox One, PlayStation 4
536886 | The Binding of Isaac: Repentance |  Action | released: 2021-03-31 | meta: nan | rating: 4.68 | platforms: PC, PlayStation 5, Xbox One, PlayStation 4, Xbox Series S/X, Nintendo Switch
545015 | Disco Elysium: Final Cut |  Adventure, RPG | released: 2021-03-30 | meta: 90 | rating: 4.66 | platforms: PC, PlayS